In [1]:
import sisl
import sisl.viz
import plotly.express as px
import os
import numpy as np
from sisl import *
import matplotlib.pyplot as plt

In [40]:
open('RUN.fdf', 'w').write("""%include STRUCT.fdf
SystemLabel siesta_2
PAO.BasisSize DZP
MeshCutoff 250. Ry
SaveHS true
SaveRho true
SaveTotalPotential true
Spin polarized
%block kgrid.MonkhorstPack
  10  1 1 0.
   1 1 1 0.
   0  0 1 0.
%endblock
%block DM.InitSpin
    1 -
    4 +
%endblock DM.InitSpin
%block ExternalElectricField
    0.000 0.500 0.000 V/Ang
%endblock ExternalElectricField
""")
rib.write('STRUCT.fdf')

open('STRUCT.fdf', 'w').write("""LatticeConstant 1.0 Ang
%block LatticeVectors
 2.45951215 0.00000000 0.00000000
 0.00000000 28.52000000 0.00000000
 0.00000000 0.00000000 14.20000000
%endblock LatticeVectors

NumberOfAtoms 10
AtomicCoordinatesFormat Ang
%block AtomicCoordinatesAndAtomicSpecies
  0.00000000 10.00000000  0.00000000 1 # 1: C
  0.00000000 12.84000000  0.00000000 1 # 2: C
  0.00000000 14.26000000  0.00000000 1 # 3: C
  0.00000000 17.10000000  0.00000000 1 # 4: C
  1.22975607 10.71000000  0.00000000 1 # 5: C
  1.22975607 12.13000000  0.00000000 1 # 6: C
  1.22975607 14.97000000  0.00000000 1 # 7: C
  1.22975607 16.39000000  0.00000000 1 # 8: C
  0.00000000 8.912000000  0.00000000 2 # 9: H
  0.00000000 18.18800000  0.00000000 2 # 10: H

%endblock AtomicCoordinatesAndAtomicSpecies

NumberOfSpecies 2
%block ChemicalSpeciesLabel
 1 6 C
 2 1 H
%endblock ChemicalSpeciesLabel

""")

861

Ara simulem amb SIESTA.

In [3]:
# Substitute path/to/your.fdf by the actual path to your fdf.
H = sisl.get_sile("/home/samba/homes/convidat/Escriptori/Dia 3/RUN.fdf").read_hamiltonian()

In [4]:
band_struct = sisl.BandStructure(
    H, # The hamiltonian that we want to diagonalize for each k point.
    points=[[0, 0, 0], [1/2, 0, 0]], # The corners of the path
    divisions=100, # The number of k points
    names=[r"\Gamma", "X"]
)
band_struct

Apliquem un camp elèctric i mirem el potencial (hauria de sortir una rampa):

In [10]:
VT=sisl.get_sile("/home/samba/homes/convidat/Escriptori/Dia 3/siesta_2.VT").read_grid()

In [11]:
VT.plot(axes="y")

Ara intenarem trobar la DOE:

In [12]:
H = sisl.get_sile("/home/samba/homes/convidat/Escriptori/Dia 3/RUN.fdf").read_hamiltonian()

In [13]:
# PDOS plot
pdos_plot = H.plot.pdos(
    kgrid=[100,1,1], nE=1000, Erange=[-1.5, 1.5],
    distribution={"method": "gaussian", "smearing": 0.1}
)

In [14]:
pdos_plot.split_DOS(on="atom+spin", name="Atom $atoms")

In [19]:
pdos_plot.update_settings(requests=[
    {"atom":0, "spin":0, "scale":-1, "name":"β", "color": "blue"},
    {"atom":0, "spin":1, "scale":-1, "name":"α", "color": "red"},
    {"atom":3, "spin":0, "scale":1, "name":"β_", "color": "blue"},
    {"atom":3, "spin":1, "scale":1, "name":"α_", "color": "red"},
])